Ejercicios
Los datos adjuntos como recurso a esta lección contienen los importes de ventas en un día en un supermercado.
1. Cree un RDD importes a partir de los datos adjuntos a esta lección como recurso. Emplee acumuladores para obtener el total de ventas realizadas y el importe total de las ventas.
2. Si se conoce que a cada venta hay que restarle un importe fijo igual a 10 pesos por temas de impuestos.
- ¿Cómo restaría este impuesto de cada venta utilizando una variable broadcast para acelerar el proceso?
- Cree un RDD llamado ventas_sin_impuestos a partir de la propuesta del inciso a que contenga las ventas sin impuestos.
- Destruya la variable broadcast creada luego de emplearla para crear el RDD del inciso b.
3. Persista el RDD ventas_sin_impuestos en los siguientes niveles de persistencia.
- Memoria.
- Disco solamente
- Memoria y disco.

In [1]:
# Instalar SDK Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
# Descargar Spark 3.4.3

!wget -q https://archive.apache.org/dist/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz

In [3]:
# Descomprimir el archivo descargado de Spark

!tar xf spark-3.4.3-bin-hadoop3.tgz

In [4]:
# establecer las variables de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.3-bin-hadoop3"

In [5]:
# Instalar la librería findspark

!pip install -q findspark

In [6]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [28]:
rdd = sc.textFile('./data/rdd.txt')


In [48]:
rdd.take(5)

['527', '386', '701', '240', '941']

In [49]:
acumulador_ventas = sc.accumulator(0)
acumulador_importe = sc.accumulator(0)

In [50]:
rdd.foreach(lambda x: acumulador_ventas.add(1))

In [51]:
rdd.foreach(lambda x: acumulador_importe.add(float(x)))

In [52]:
print(acumulador_ventas)

10000


In [53]:
print(acumulador_importe)

5042335.0


In [100]:
porcentaje = 0.1

In [101]:
impuesto = sc.broadcast(porcentaje)

In [102]:
rdd_sinImpuestos = rdd.map(lambda x: float(x)-(float(x)*impuesto.value))

In [103]:
rdd_sinImpuestos.take(5)

[474.3, 347.4, 630.9, 216.0, 846.9]

In [104]:
impuesto.destroy()

In [105]:
from pyspark.storagelevel import StorageLevel
rdd_sinImpuestos.persist(StorageLevel.MEMORY_ONLY)

PythonRDD[40] at RDD at PythonRDD.scala:53

In [106]:
rdd_sinImpuestos.unpersist()
rdd_sinImpuestos.cache()

PythonRDD[40] at RDD at PythonRDD.scala:53

In [58]:
rdd_sinImpuestos.unpersist()
rdd_sinImpuestos.persist(StorageLevel.DISK_ONLY)

PythonRDD[23] at collect at <ipython-input-57-1e76db63ec29>:1

In [107]:
rdd_sinImpuestos.unpersist()
rdd_sinImpuestos.persist(StorageLevel.MEMORY_AND_DISK)

PythonRDD[40] at RDD at PythonRDD.scala:53